In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip uninstall tensorflow -y

In [ ]:
!pip install tensorflow==1.14.0

In [ ]:
!pip uninstall keras -y

In [ ]:
!pip install keras==2.3.0

In [ ]:
!pip install shap==0.30.1

In [ ]:
import tensorflow as tf
import keras
import shap

In [ ]:
print(keras.__version__)
print(tf.__version__)
print(shap.__version__)

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import nltk
from nltk.corpus import stopwords
from keras.utils import to_categorical
from sklearn import preprocessing

In [ ]:
data = pd.read_csv('../input/news-extracted-dataset/News_category_extract_dataset.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)
data.shape

In [ ]:
data = data.sample(frac=0.2, random_state=99).reset_index(drop=True)
data = shuffle(data, random_state=22)
data = data.reset_index(drop=True)
data.shape

In [ ]:
from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 200
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['clean_text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(data['clean_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
le = preprocessing.LabelEncoder()
labels=data['category'].values
labels_clf=le.fit_transform(labels)
classes=le.classes_

In [ ]:
classes

In [ ]:
cat_labels=labels_clf
cat_labels = to_categorical(cat_labels)

In [ ]:
cat_labels

In [ ]:
from sklearn.model_selection import train_test_split
Y=cat_labels
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(X_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
from keras.callbacks import EarlyStopping
epochs = 20
batch_size = 64
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
# select a set of samples to take an expectation over
distrib_samples = X_train[:100]
session = keras.backend.tensorflow_backend.get_session()
# session had to be manually specified
# otherwise looked for Keras.._SESSION ct. which doesn't exist!
explainer = shap.DeepExplainer(model, distrib_samples, session)
num_explanations = 10

In [ ]:
shap_values = explainer.shap_values(X_test[:num_explanations])
print('Shape of shap_values:', np.shape(shap_values))
shap_values

In [ ]:
num2word = {}
for w in word_index.keys():
    num2word[word_index[w]] = w
x_test_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), X_test[i]))) for i in range(10)])
shap.summary_plot(shap_values, feature_names = list(num2word.values()), class_names = list(classes))

In [ ]:
word_index

In [ ]:
num2word

In [ ]:
x_test_words

In [ ]:
kc2=[]
def ABS_SHAP(df_shap,df):
    global kc2
    #import matplotlib as plt
    # Make a copy of the input data
    shap_v = pd.DataFrame(df_shap)
    df=pd.DataFrame(df_shap)
    feature_list = df.columns
    shap_v.columns = feature_list
    df_v = df.copy().reset_index().drop('index',axis=1)
   
    # Determine the correlation in order to plot with different colors
    corr_list = list()
    for i in feature_list:
        b = np.corrcoef(shap_v[i],df_v[i])[1][0]
        corr_list.append(b)
    corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
    # Make a data frame. Column 1 is the feature, and Column 2 is the correlation coefficient
    corr_df.columns  = ['Variable','Corr']
    corr_df['Sign'] = np.where(corr_df['Corr']>0,'red','blue')
   
    # Plot it
    shap_abs = np.abs(shap_v)
    k=pd.DataFrame(shap_abs.mean()).reset_index()
    k.columns = ['Variable','SHAP_abs']
    kc2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = kc2.sort_values(by='SHAP_abs',ascending = True)
    print(k2)
    colorlist = k2['Sign']
    ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, figsize=(5,6),legend=False)
    ax.set_xlabel("SHAP Value (Red = Positive Impact)")

In [ ]:
def convertor(sequence):   
    converted_text_list=[]
    for val in sequence:
        if val!=0:
            converted_text_list.append(vocabulary[val])
    converted_text=' '.join(converted_text_list)
    return converted_text

In [ ]:
d=dict(zip([0,1,2,3],list(classes)))

In [ ]:
tokens=list(word_index.keys())
values=list(word_index.values())
vocabulary=dict(zip(values,tokens))

In [ ]:
idx=9
y_preds = model.predict(X_test)
print(X_test[idx])
print(convertor(X_test[idx]))
print('Actual_class:',d[np.argmax(Y_test[idx])],np.argmax(Y_test[idx]))
print('pred_class:',d[np.argmax(y_preds[idx])],np.argmax(y_preds[idx]))

In [ ]:
shap.initjs()
class_num = np.argmax(y_preds[idx])
shap.force_plot(explainer.expected_value[class_num], shap_values[class_num][idx], x_test_words[idx])

In [ ]:
plots=[]
for idx in range(0,10):
    print('sample number: ',idx)
    y_preds = model.predict(X_test)
    print(convertor(X_test[idx]))
    print('Actual_class:',d[np.argmax(Y_test[idx])],np.argmax(Y_test[idx]))
    print('pred_class:',d[np.argmax(y_preds[idx])],np.argmax(y_preds[idx]))
    class_num = np.argmax(y_preds[idx])
    display(shap.force_plot(explainer.expected_value[class_num], shap_values[class_num][idx], x_test_words[idx]))


In [ ]:
idx=5
plots=[]
print('sample number: ',idx)
y_preds = model.predict(X_test)
print(convertor(X_test[idx]))
print('Actual_class:',d[np.argmax(Y_test[idx])],np.argmax(Y_test[idx]))
print('pred_class:',d[np.argmax(y_preds[idx])],np.argmax(y_preds[idx]))
for class_num in range(0,4):
    print('force plot for class',class_num,':',d[class_num])
    display(shap.force_plot(explainer.expected_value[class_num], shap_values[class_num][idx], x_test_words[idx]))